### 3. post-processing result ner

In [ ]:
# !pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/NER/NER_Topic4/Analyze_Result

/content/drive/.shortcut-targets-by-id/1_PyaczLRAlu5N-sh8Vrx70SdkU4ZXwH6/NER/NER_Topic4/Analyze_Result


In [ ]:
ls

1_NER_topic4_Result_Extract.ipynb
2_NER_topic4_Result_Analyze_test1.ipynb
recoginzed_NER_topic4.csv
recoginzed_NER_topic4_distilbert_89_256seg_40ep_balanced_5000sents.csv
recoginzed_NER_topic4_finalModel.csv
taken_ner_result.csv
taken_ner_result_distilbert_89_256seg_40ep_balanced_5000sents.csv
taken_words_with_rsp_types.csv
taken_words_with_rsp_types_distilbert_89_256seg_40ep_balanced_5000sents.csv
topic4_30000sents_ner_finalModel.csv
topic4_30000sents_step1_explodedData.csv
topic4_ner_32_distilbert_5000sents.csv
topic_4_ner_32_distilbert.csv
topic4_ner_32_distilbert_sentcorpus.csv
topic4_ner_distilbert_89_256seg_40ep_balanced_5000sents.csv
topic4_ner_distilbert_finalModel_5000sents.csv


In [ ]:
import torch
import pandas as pd
import time
import ast
start_time = time.time()

In [ ]:
data = pd.read_csv("topic4_30000sents_step1_explodedData.csv",encoding="latin1" )

In [ ]:
data.head()

,Unnamed: 0,doc_id,sent_id,word_id,word,entity_type
0,2,0,0,2,retinal,I-DISEASE_OR_SYNDROME
1,3,0,0,3,vein,I-DISEASE_OR_SYNDROME
2,7,0,0,7,age-related,B-GENE_OR_GENOME
3,8,0,0,8,macular,B-TISSUE
4,9,0,0,9,degeneration,I-DISEASE_OR_SYNDROME


In [ ]:
print('length of data: ', len(data))

length of data:  151825


In [ ]:
data = data.dropna()

In [ ]:
print('length of data: ', len(data))

length of data:  151822


In [ ]:
data = data.iloc[: , 1:]

In [ ]:
data['begin tag'] = data['entity_type'].apply(lambda x: x[:1])

In [ ]:
data['type'] = data['entity_type'].apply(lambda x: x[2:])

In [ ]:
data.head()

,doc_id,sent_id,word_id,word,entity_type,begin tag,type
0,0,0,2,retinal,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME
1,0,0,3,vein,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME
2,0,0,7,age-related,B-GENE_OR_GENOME,B,GENE_OR_GENOME
3,0,0,8,macular,B-TISSUE,B,TISSUE
4,0,0,9,degeneration,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME


In [ ]:
I_idx = data[data['begin tag'] == 'I'].index.tolist()

In [ ]:
B_idx = data[data['begin tag'] == 'B'].index.tolist()

In [ ]:
data['Note'] = ''

In [ ]:
merged_B_list = []

for idx in I_idx:
  if ((idx-1) in B_idx) and (data.loc[idx, 'type'] == data.loc[idx-1, 'type']):
    data.loc[idx, 'Note'] = 'to merge with prev word'
    
    # drop taken B from the dataset
    merged_B_list.append(idx-1)

In [ ]:
print('Data to be merged: ', len(merged_B_list))

Data to be merged:  8617


In [ ]:
data.head()

,doc_id,sent_id,word_id,word,entity_type,begin tag,type,Note
0,0,0,2,retinal,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME,
1,0,0,3,vein,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME,
2,0,0,7,age-related,B-GENE_OR_GENOME,B,GENE_OR_GENOME,
3,0,0,8,macular,B-TISSUE,B,TISSUE,
4,0,0,9,degeneration,I-DISEASE_OR_SYNDROME,I,DISEASE_OR_SYNDROME,


In [ ]:
merge_df = pd.DataFrame()

In [ ]:
for size_idx in range(len(merged_B_list)):
  merge_df.loc[size_idx, 'doc_id'] = int(data.loc[merged_B_list[size_idx], 'doc_id'])
  merge_df.loc[size_idx, 'sent_id'] = int(data.loc[merged_B_list[size_idx], 'sent_id'])
  merge_df.loc[size_idx, 'word_id'] = int(data.loc[merged_B_list[size_idx], 'word_id'])
  merge_df.loc[size_idx, 'type'] = data.loc[merged_B_list[size_idx], 'type']

  word = data.loc[merged_B_list[size_idx], 'word'] + ' ' + data.loc[merged_B_list[size_idx]+1, 'word']
  merge_df.loc[size_idx, 'word'] = word

In [ ]:
merge_df.head()

,doc_id,sent_id,word_id,type,word
0,0.0,3.0,4.0,DISEASE_OR_SYNDROME,submacular hemorrhage
1,0.0,5.0,4.0,DISEASE_OR_SYNDROME,ocular disease
2,0.0,9.0,14.0,DISEASE_OR_SYNDROME,retinal hemorrhage
3,1.0,0.0,11.0,CARDINAL,over 200
4,1.0,6.0,21.0,GENE_OR_GENOME,anti-sars-cov-2 antibody


In [ ]:
# remove all the data related to merged data 'B' tag

data.drop(merged_B_list, axis=0, inplace=True)

In [ ]:
print('length of data: ', len(data))

length of data:  143205


In [ ]:
one_word_df_rough = data[data['begin tag'] == 'B']

In [ ]:
one_word_df = one_word_df_rough[['doc_id', 'sent_id', 'word_id', 'type', 'word']]

In [ ]:
one_word_df.head()

,doc_id,sent_id,word_id,type,word
2,0,0,7,GENE_OR_GENOME,age-related
3,0,0,8,TISSUE,macular
7,0,0,17,CORONAVIRUS,sars-cov-2
8,0,0,18,CHEMICAL,"vaccines,"
9,0,1,3,GENE_OR_GENOME,fundus


In [ ]:
# taken dataset
# merge B only dataset + merged dataset = taken dataset

taken_df = pd.concat([merge_df, one_word_df])

In [ ]:
taken_df['doc_id'] = taken_df['doc_id'].fillna(0).astype(int)
taken_df['sent_id'] = taken_df['sent_id'].fillna(0).astype(int)
taken_df['word_id'] = taken_df['word_id'].fillna(0).astype(int)

In [ ]:
taken_df.head()

,doc_id,sent_id,word_id,type,word
0,0,3,4,DISEASE_OR_SYNDROME,submacular hemorrhage
1,0,5,4,DISEASE_OR_SYNDROME,ocular disease
2,0,9,14,DISEASE_OR_SYNDROME,retinal hemorrhage
3,1,0,11,CARDINAL,over 200
4,1,6,21,GENE_OR_GENOME,anti-sars-cov-2 antibody


In [ ]:
print('Merged df len: ', len(merge_df))
print('One Word df len: ', len(one_word_df))
print('Taken df len: ', len(taken_df))

Merged df len:  8617
One Word df len:  103834
Taken df len:  112451


In [ ]:
taken_df.to_csv('topic4_30000sents_entityData_finalModel.csv')

In [ ]:
# extract taken entity types by respective groups

analyze_df = pd.DataFrame()

detected_types = list(set(taken_df['type'].tolist()))

for e_type in detected_types:
  word_lst = taken_df[taken_df['type'] == e_type]['word'].tolist()
  analyze_df[e_type] = pd.Series(word_lst)

In [ ]:
analyze_df.to_csv('taken_words_with_rsp_types_30000sents_finalModel.csv')

Removed Data Analysis

In [ ]:
# removed dataset = original - (B only + Merge_B_index + (Merge_B_index + 1))
'''
data.drop(one_word_df.index.tolist(), axis=0, inplace=True)

#data.drop(merged_B_list, axis=0, inplace=True)

merged_I_list = [b+1 for b in merged_B_list]
data.drop(merged_I_list, axis=0, inplace=True)
'''

'\ndata.drop(one_word_df.index.tolist(), axis=0, inplace=True)\n\n#data.drop(merged_B_list, axis=0, inplace=True)\n\nmerged_I_list = [b+1 for b in merged_B_list]\ndata.drop(merged_I_list, axis=0, inplace=True)\n'

In [ ]:
end_time = time.time()

print('Estimate runtime: ', (end_time-start_time)/60, ' minutes.')

Estimate runtime:  1.240830628077189  minutes.


In [ ]:
# for 30000 sents

# Estimate runtime:   1.240830628077189  minutes.